In [ ]:
# Load the data
data_path = r'C:\Users\LOQ\OneDrive\Pose Health Care\MA (PPop)\สถิติ Pose Repairman.xlsx'
df = pd.read_excel(data_path, sheet_name="ข้อมูลการใช้นำยา")


In [ ]:
df
df = df.dropna()


In [ ]:
df

,แผนก,หมายเลขเครื่อง,ปัญหา,ระยะเวลาในใช้น้ำยา /แบต (วัน)
0,WARD 5C,2,น้ำยาหมด,23
1,WARD 5C,3,น้ำยาหมด,28
2,WARD 5C,3,น้ำยาหมด,30
3,WARD 5C,2,น้ำยาหมด,40
4,WARD 5C,3,น้ำยาหมด,25
...,...,...,...,...
1962,KIDNEY(ศูนย์โรคไต),1473,แบตเตอร์รี่หมด,153
1963,KIDNEY(ศูนย์โรคไต),1474,แบตเตอร์รี่หมด,169
1964,KIDNEY(ศูนย์โรคไต),1475,แบตเตอร์รี่หมด,156
1965,KIDNEY(ศูนย์โรคไต),1476,แบตเตอร์รี่หมด,157


In [ ]:
# Display DataFrame columns for debugging
st.write("DataFrame Columns:")
st.write(df.columns)

In [ ]:
# Define column names
department_column = 'แผนก'
machine_id_column = 'หมายเลขเครื่อง'
issue_column = 'ปัญหา'
maintenance_duration_column = 'ระยะเวลาในใช้น้ำยา /แบต (วัน)'


In [ ]:
# Ensure correct column names and preprocessing
if maintenance_duration_column in df.columns:
    df[maintenance_duration_column] = pd.to_numeric(df[maintenance_duration_column], errors='coerce')
    df.dropna(subset=[maintenance_duration_column], inplace=True)
else:
    st.error(f"Column '{maintenance_duration_column}' not found in the data.")
    st.stop()

In [ ]:
# Streamlit app
st.title("Machinery Maintenance Information and Prediction")

DeltaGenerator()

In [ ]:
# Display the data
st.header("Maintenance Records")
st.table(df)

DeltaGenerator()

In [ ]:

# One-hot encode the categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(df[[department_column, issue_column]])



In [ ]:

# Create a DataFrame with the encoded features
encoded_feature_names = encoder.get_feature_names_out([department_column, issue_column])
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names)


In [ ]:

# Combine the encoded features with the numeric feature
X = pd.concat([df[[machine_id_column]], encoded_df], axis=1)
y = df[maintenance_duration_column]
